In [2]:
!pip install pandas
!pip install sentence-transformers
!pip install accelerate
!pip install -U bitsandbytes
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 194.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 193.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 178.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 59.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 85.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 154.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 78.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.9/789.9 kB 78.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 108.1 MB/s  0:00:040:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 141.0 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 174.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB

In [2]:
# File paths
input_file = "DATASET6/final.csv"
output_file = "DATASET6/final_with_header.csv"

# Your header as a string, comma-separated
header = "label,text"

# Open the original file for reading, and a new file for writing
with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
    f_out.write(header + '\n')  # Write header first
    for line in f_in:            # Copy rest of the file line by line
        f_out.write(line)


In [20]:
import pandas as pd
df = pd.read_csv('DATASET6/final_with_header.csv')
laliste=[]
laliste_labels=[]
for elt in df['label']:
    laliste_labels.append(elt)
for elt in df['text']:
    laliste.append(elt)

In [38]:
def equal_distribution(data,labels, amount):
    new_dataset={}
    label_counter={}
    for label in labels:
        label_counter[label]=0
        new_dataset[label]=[]
    for label,text in zip(labels,data):
        if label_counter[label]<amount:
            new_dataset[label].append(text)
            label_counter[label]+=1
    return new_dataset


In [39]:
new_dataset=equal_distribution(laliste,laliste_labels,10)

In [47]:
len(new_dataset[5])

10

In [6]:
from sentence_transformers import SentenceTransformer
import torch

# Define arguments for 4-bit quantization
model_kwargs = {
    "device_map": "auto",         # Automatically map layers to devices (GPU/CPU)
    "torch_dtype": torch.bfloat16,  # Recommended dtype for Qwen models
    "load_in_4bit": True,           # Enable 4-bit quantization
}

# The model card also recommends enabling flash_attention_2 for better performance
# You may need to install it: pip install flash-attn
# Set use_flash_attention_2=True if available
embedding_model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-8B",
    model_kwargs=model_kwargs,
    trust_remote_code=True, # Often required for custom model architectures
    # use_flash_attention_2=True, # Uncomment if flash-attn is installed
)

/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.31s/it]


In [7]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

Generating test split: 100%|██████████| 139/139 [00:00<00:00, 5956.48 examples/s]


In [ ]:
laliste_labels=[]
for elt in train['intent']:
    laliste_labels.append(elt)

In [8]:
documents = laliste[:10]

len(documents)

10

In [12]:
documents = laliste

synth_data = torch.from_numpy(embedding_model.encode(documents))

In [13]:


#synth_data = torch.from_numpy(embedding_model.encode(train['cleaned_cite_text'].tolist()))
validation_data = torch.from_numpy(embedding_model.encode(validation['cleaned_cite_text'].tolist()))
test_data = torch.from_numpy(embedding_model.encode(test['cleaned_cite_text'].tolist()))

In [15]:
import torch

import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=128, num_classes=6):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)
mymodel = MLPClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=0.001)

In [16]:
from torch.utils.data import TensorDataset, DataLoader
X_train = synth_data
y_train = torch.tensor(laliste_labels)

batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [17]:
num_epochs = 10

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()           # ✅ Reset gradients from previous step
        outputs = mymodel(X_batch)        # Forward pass
        loss = criterion(outputs, y_batch)
        loss.backward()                 # Compute gradients
        optimizer.step()                # Update weights


In [18]:
from sklearn.metrics import classification_report
with torch.no_grad():
    validation_outputs = mymodel(validation_data)
    test_outputs = mymodel(test_data)

In [19]:
torch.argmax(validation_outputs,axis=1)

tensor([1, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 4, 2, 2, 1, 1,
        1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
        2, 1, 1, 2, 4, 2, 1, 5, 1, 1, 0, 2, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 1, 5, 2, 1, 2, 5, 5, 5, 1, 1, 1, 1, 2])

In [20]:
print(classification_report(validation['intent'], torch.argmax(validation_outputs, axis=1)))

              precision    recall  f1-score   support

           0       1.00      0.02      0.03        59
           1       0.25      1.00      0.40        21
           2       0.67      0.48      0.56        21
           3       0.25      0.25      0.25         4
           4       0.00      0.00      0.00         5
           5       0.43      0.75      0.55         4

    accuracy                           0.32       114
   macro avg       0.43      0.42      0.30       114
weighted avg       0.71      0.32      0.22       114

